In [ ]:
from keras.layers import *
from keras import regularizers
from keras.models import Model
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K

import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join
import imageio
from IPython.display import Image
import plot
import importlib

importlib.reload(plot)

import numpy as np

## What are autoencoders

* Multilayer artificial neural networks
* Uses `representation learning`
* Tries to mimic input at the output
<img src="http://fastforwardlabs.github.io/blog-images/miriam/miriams-figure.png">

### * Map high-dimensional data another space
### * Compression
### * Learn abstract features in an unsupervised way (labeled data is expensive)
### * Denoising and hole-filling
<img src="hole-fill.png">

## Basic autoencoder example


In [ ]:
(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

### Build model

In [ ]:
from keras import regularizers

encoding_dim = 32
input_img = Input(shape=(28 * 28,))
encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-50))(input_img)
# encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(28 * 28, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# get encoder and decoder separately
encoded_input = Input(shape=(encoding_dim, ))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
encoder = Model(input_img, encoded)


### Train model

In [ ]:
importlib.reload(plot)
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[plot.Plot(x_test, y_test, encoder, decoder, 'AE')])

In [ ]:
IMAGES_PATH = 'images/'
GIFS_PATH = 'gifs/'

filenames = [IMAGES_PATH + f for f in listdir(IMAGES_PATH) if isfile(join(IMAGES_PATH, f))]
reconstruction_images = [imageio.imread(filename) for filename in filenames if 'reconstruction' in filename]
pca_ica_images = [imageio.imread(filename) for filename in filenames if 'reconstruction' not in filename]
imageio.mimsave(GIFS_PATH + 'reconstruction.gif', reconstruction_images)
imageio.mimsave(GIFS_PATH + 'pca_ica.gif', pca_ica_images)

In [ ]:
Image(filename=GIFS_PATH + 'reconstruction.gif')

In [ ]:
Image(filename=GIFS_PATH + 'pca_ica.gif')

In [ ]:
number_of_images = 10
encoded_imgs = encoder.predict(x_test[:number_of_images])
decoded_imgs = decoder.predict(encoded_imgs)

plt.figure(figsize=(20, 4))
for i in range(number_of_images):
    # display original
    ax = plt.subplot(2, number_of_images, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, number_of_images, i + 1 + number_of_images)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


## Variational Autoencoders

- Probabalistic approach to autoencoders
- learns a Latent Variable Model Instead of learning an arbitrary representation
- Generally no parameter tuning required

<img src="https://cdn-images-1.medium.com/max/2000/1*22cSCfmktNIwH5m__u2ffA.png">

- Vanilla autoencoder is deterministic
- Variational Autoencoder is stochastic. It learns a lattent variable $z$ from inputs $x$

> Probabilistic encoder approximating the true posterior $q(z|x)$.
***
> Generative decoder samples posterior $\hat{q}(z|x)$ aproximation, which does not rely on any particular input x.

- Learning conditional distributions is facilitated by forcing a prior.
$$  z\sim N(0,I). $$
<img src="http://fastforwardlabs.github.io/blog-images/miriam/imgs_code/vae.4.png">


### A VAE has three basic parts:

- 1) An encoder that that learns the parameters (mean and variance) of the underlying latent distribution;
- 2) A means of sampling from that distribution;
- 3) A decoder that can turn the sample back into an image.

### 1) Encoder:

In [ ]:
img_size = (28 * 28, )
batch_size = 256
hidden_size = 128
latent_size = 16

inputs = Input(shape=img_size, name='encoder_input')
x = Dense(hidden_size, activation='relu')(inputs)
z_mean = Dense(latent_size, name='z_mean')(x)
z_log_var = Dense(latent_size, name='z_log_var')(x)

### 2) Sampling function

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    batch_size = K.shape(z_mean)[0]
    latent_size = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch_size, latent_size))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_size,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

### 3) Decoder

In [ ]:
latent_inputs = Input(shape=(latent_size,), name='z_sampling')
x = Dense(hidden_size, activation='relu')(latent_inputs)
outputs = Dense(28 * 28, activation='sigmoid')(x)
decoder = Model(latent_inputs, outputs, name='decoder')
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

reconstruction_loss = binary_crossentropy(inputs, outputs)
reconstruction_loss *= (28 * 28)
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

vae.compile(optimizer='adam')
vae.summary()

### Train network

In [ ]:
idx = np.arange(5000)
np.random.shuffle(idx)

vae.fit(x=x_train, y=None,
        shuffle=True,
        epochs=50,
        batch_size=256,
        validation_data=(x_test, None))

In [ ]:
X_valid_noTest = x_test.reshape(-1,28,28,1)
# Translate into the latent space
encoder = Model(input_img, z_mu)
x_valid_noTest_encoded = encoder.predict(X_valid_noTest, batch_size=batch_size)
plt.figure(figsize=(10, 10))
plt.scatter(x_valid_noTest_encoded[:, 0], x_valid_noTest_encoded[:, 1], c=y_test, cmap='brg')
plt.colorbar()
plt.show()
